<h1 style=\"text-align: center; font-size: 50px;\"> Pre-trained BERT for building a Q&A system MLflow integration </h1>

Notebook Overview
- Start Execution
- Install and Import Libraries
- Configure Settings
- Logging Model to MLflow
- Fetching the Latest Model Version from MLflow
- Loading the Model and Running Inference

## Start Execution

In [ ]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("register_model_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [ ]:
start_time = time.time()  

logger.info("Notebook execution started.")

## Install and Import Libraries

In [ ]:
# ------------------------ Standard Library Imports ------------------------
import warnings
from pathlib import Path
import shutil

# ------------------------ MLflow for Experiment Tracking and Model Management ------------------------
import mlflow
from mlflow.types.schema import Schema, ColSpec
from mlflow.types import ParamSchema, ParamSpec
from mlflow.models import ModelSignature
from mlflow import MlflowClient

from transformers import  pipeline


## Configure Settings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
# ------------------------- Define global experiment and run names to be used throughout the notebook -------------------------

MODEL_CHECKPOINT = "distilbert-base-cased"
EXPERIMENT_SET = "BERT Q&A - distilbert-base-cased"
SAVE_MODEL_NAME = "../distilbert_bertqa"
EXPERIMENT_NAME = "BERT model for Q&A"
MODEL_NAME = "BERT_QA"
RUN_NAME = 'BERT_QA'
NAME = 'BERT_QA'

# ------------------------- Set up the chunk separator for text processing -------------------------
CHUNK_SEPARATOR = "\n\n"

## Logging Model to MLflow

In [ ]:
model_name = SAVE_MODEL_NAME
qa_pipeline = pipeline(
    'question-answering',
    model = model_name,
    device=0 #GPU
)

class DistilBERTModel(mlflow.pyfunc.PythonModel):
    def _preprocess(self, inputs):
        """
        Preprocesses the input data.

        Args:
            inputs: A dictionary containing two keys:
                - 'context': A list with the context text.
                - 'question': A list with the question to be answered.

        Returns:
            tuple: A tuple containing the context (str) and the question (str).
        """
        try:
                context = inputs['context'][0]
                question = inputs['question'][0]
                print("pre processing", context,question)
                return context, question
        
        except Exception as e:
            logger.error(f"Error preprocessing the input data: {str(e)}")  

    def load_context(self, context):
        """
        Loads the question-answering pipeline using the saved model artifact.

        Args:
            context: The MLflow context object 
                containing the loaded artifacts.
        """
        try:
            self.model = pipeline(
            'question-answering',
             model=context.artifacts["model"],
             device=0
        )
        except Exception as e:
            logger.error(f"Error loading the question-answering pipeline: {str(e)}")     

    def predict(self, context, model_input, params):
        """
        Runs inference using the loaded model and input data.

        Args:
            context: The MLflow context object 
                with access to artifacts.
            model_input: A dictionary containing 'context' and 'question' keys.

        Returns:
            The output from the model containing the predicted answer and optionally the score.
        """
        try:
            in_ctx, question = self._preprocess(model_input)
            output = self.model(context=in_ctx, question=question)
            return output
        except Exception as e:
            logger.error(f"Error running inference: {str(e)}")  

    @classmethod
    def log_model(cls, model_name, source_trainer = None, source_pipeline = None, demo_folder="../demo"):
        """
        Logs the model to MLflow, including artifacts, dependencies, and input/output signatures.

        Args:
            model_name: Path where the model will be temporarily saved before logging.
            source_trainer: A trainer object with a `.save_model()` method. Defaults to None.
            source_pipeline: A pipeline object with a `.save_pretrained()` method. Defaults to None.
            demo_folder: Path to the folder containing the compiled demo UI. Defaults to "demo".
        """
        try:
            input_schema = Schema(
            [
                ColSpec("string", "context"),
                ColSpec("string", "question"),
            ]
            )
            output_schema = Schema(
                [
                    ColSpec("string", "answer")
                ]
            )
            
            params_schema = ParamSchema(
                [
                    ParamSpec("show_score", "boolean", False)
                ]
            )
          
            signature = ModelSignature(inputs=input_schema, outputs=output_schema, params=params_schema)
            if source_trainer is not None:
                source_trainer.save_model(model_name)
            elif source_pipeline is not None:
                source_pipeline.save_pretrained(model_name)
                 
            requirements = [
                "transformers==4.48.0",
                "tf_keras"
            ]
            mlflow.pyfunc.log_model(
                model_name,
                python_model=cls(),
                artifacts={"model": model_name, "demo": demo_folder},
                signature=signature,
                pip_requirements=requirements
            )
            shutil.rmtree(model_name)
            logger.info("Logging model to MLflow done successfully")

        except Exception as e:
            logger.error(f"Error logging model to MLflow: {str(e)}")

In [ ]:
mlflow.set_tracking_uri('/phoenix/mlflow')
mlflow.set_experiment(experiment_name = EXPERIMENT_NAME)
with mlflow.start_run(run_name= RUN_NAME) as run:
    logger.info(f"Run's Artifact URI: {run.info.artifact_uri}")
    DistilBERTModel.log_model(model_name = MODEL_NAME, source_pipeline=qa_pipeline)
    mlflow.register_model(model_uri = f"runs:/{run.info.run_id}/{MODEL_NAME}", name = NAME)

## Fetching the Latest Model Version from MLflow

In [ ]:
client = mlflow.MlflowClient()
model_metadata = client.get_latest_versions(MODEL_NAME, stages=["None"])
latest_model_version = model_metadata[0].version
print(latest_model_version, mlflow.models.get_model_info(f"models:/BERT_QA/{latest_model_version}").signature)

## Loading the Model and Running Inference

In [ ]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/BERT_QA/{latest_model_version}")
context = "Marta is mother of John and Amanda"
question = "what is the name of Marta's daugther?"
model.predict({"context": [context], "question":[question]})

In [ ]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

Built with ❤️ using Z by HP AI Studio.